# Will It Blend? Deploying BlenderBot

First, load the model.

In [1]:
from parlai.core.agents import create_agent_from_model_file
blender_agent = create_agent_from_model_file("blender_distilled/blender_distilled.pth.checkpoint",
                                            opt_overrides={
                                                "model": "transformer/generator",
                                                "skip_generation": False,
                                                "topk": 100,
                                                "inference": "topk",
                                                "temperature": 0.7,
                                            })



06:23:49 | Overriding opt["model"] to transformer/generator (previously: projects.anti_scaling.distillation:DistillTransformerAgent)
06:23:49 | Overriding opt["skip_generation"] to False (previously: True)
06:23:49 | Overriding opt["topk"] to 100 (previously: 10)
06:23:49 | Overriding opt["inference"] to topk (previously: beam)
06:23:49 | Overriding opt["temperature"] to 0.7 (previously: 1.0)
06:23:49 | Using CUDA
06:23:49 | loading dictionary from blender_distilled/blender_distilled.pth.checkpoint.dict
06:23:49 | num words = 8008
06:24:11 | Total parameters: 1,122,675,200 (1,122,019,840 trainable)
06:24:11 | Loading existing model params from blender_distilled/blender_distilled.pth.checkpoint


In [25]:
blender_agent.opt["beam_min_length"] = 50


## Example

Minimal example from Github issue

In [ ]:
from itertools import zip_longest
import time
from parlai.utils.strings import normalize_reply

start = time.time()

blender_agent.reset()

history = [
    "hi, where are you from",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah",
    "that's great, what is your favorite place?",
    "i think i like"
]

def process(history):
    pairs = zip_longest(*[iter(history)]*2)

    for user, bot in pairs:
        blender_agent.observe({'text': user, 'episode_done': False})
        if bot:
            blender_agent.history.add_reply(bot)

    from IPython.core.debugger import set_trace; set_trace()
    response = blender_agent.act()
    elapsed = time.time() - start
    print(f"BlenderBot replied in {elapsed * 1000:.0f}ms: {normalize_reply(response['text'])}")
    print("All beams:", "\n".join([t for t, s in response["beam_texts"]]))
    print()

    print("BlenderBot's history view:")
    print(blender_agent.history)
    
process(history)

> <ipython-input-9-a3d1cd5ed173>(26)process()
     24 
     25     from IPython.core.debugger import set_trace; set_trace()
---> 26     response = blender_agent.act()
     27     elapsed = time.time() - start
     28     print(f"BlenderBot replied in {elapsed * 1000:.0f}ms: {normalize_reply(response['text'])}")

ipdb> b parlai/core/torch_generator_agent.py:890
Breakpoint 3 at /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py:890
ipdb> c
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(890)eval_step()
    888         else:
    889             maxlen = self.label_truncate or 256
3-> 890             beam_preds_scores, beams = self._generate(batch, self.beam_size, maxlen)
    891             preds, scores = zip(*beam_preds_scores)
    892             self._add_generation_metrics(batch, preds)

ipdb> step
--Call--
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1047)_generate()
   1045         return decode

ipdb> prefix_tokens = torch.LongTensor(self.history.parse("i think that")).cuda().unsqueeze(0)
ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1087)_generate()
   1085 
   1086         bsz = batch.batchsize
-> 1087         if batch.text_vec is not None:
   1088             batchsize = batch.batchsize
   1089             beams = [

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1088)_generate()
   1086         bsz = batch.batchsize
   1087         if batch.text_vec is not None:
-> 1088             batchsize = batch.batchsize
   1089             beams = [
   1090                 self._treesearch_factory(dev)

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1090)_generate()
   1088             batchsize = batch.batchsize
   1089             beams = [
-> 1090                 self._treesearch_factory(dev)
   1091                 .set_context(self._get_context(batch, batch_idx))

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1106)_generate()
   1104 
   1105         for _ts in range(max_ts):
-> 1106             if all((b.is_done() for b in beams)):
   1107                 # exit early if possible
   1108                 break

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1110)_generate()
   1108                 break
   1109 
-> 1110             score, incr_state = model.decoder(decoder_input, encoder_states, incr_state)
   1111             # only need the final hidden state to make the word prediction
   1112             score = score[:, -1:, :]

ipdb> len(beams)
1
ipdb> beams
ipdb> beam_size
10
ipdb> decoder_input 
tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], device='cuda:0')
ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1112)_generate()
   1110           

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1120)_generate()
   1118             # force to fp32 to avoid overflow issues during search calculations
   1119             score = F.log_softmax(score, dim=-1, dtype=torch.float32)  # type: ignore
-> 1120             if prefix_tokens is not None and _ts < prefix_tokens.size(1):
   1121                 # generate prefix_tokens for every timestep that they exist
   1122                 # achieve by setting score of all other tokens to be -inf

ipdb> prefix_tokens 
tensor([[607, 507, 319]], device='cuda:0')
ipdb> _ts
0
ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1123)_generate()
   1121                 # generate prefix_tokens for every timestep that they exist
   1122                 # achieve by setting score of all other tokens to be -inf
-> 1123                 prefix_toks = prefix_tokens[:, _ts].unsqueeze(-1).repeat(1, beam_size)
   1124                 pr

ipdb> prefix_mask 
tensor([[True, True, True, True, True, True, True, True, True, True]],
       device='cuda:0')
ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1129)_generate()
   1127                 score[prefix_mask] = score[prefix_mask].scatter_(
   1128                     -1,
-> 1129                     prefix_toks[prefix_mask].unsqueeze(-1),
   1130                     prefix_score[prefix_mask],
   1131                 )

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1130)_generate()
   1128                     -1,
   1129                     prefix_toks[prefix_mask].unsqueeze(-1),
-> 1130                     prefix_score[prefix_mask],
   1131                 )
   1132             for i, b in enumerate(beams):

ipdb> score
tensor([[[-1.0000e+20, -1.0000e+20, -1.0000e+20,  ..., -1.0000e+20,
          -1.0000e+20, -1.0000e+20],
         [-1.0000e+20, -1.0000e+20, -1.0000e+20,  ..., -1.0000e+20,
       

ipdb> n
> /home/ubuntu/efs/tchang97/repos/ParlAI/parlai/core/torch_generator_agent.py(1141)_generate()
   1139                 ]
   1140             )
-> 1141             incr_state = model.reorder_decoder_incremental_state(
   1142                 incr_state, incr_state_inds
   1143             )



In [27]:
response

{'id': 'TransformerGenerator',
 'episode_done': False,
 'text': 'i think it is a little too spicy for me, but i love the beach. ',
 'beam_texts': [('i think it is a little too spicy for me, but i love the beach. ',
   -10.656795501708984),
  ("i've never been there! do you have any pets? i have a dog, she's a beagle named oreo.",
   -11.309974670410156),
  ('I have never been there! What is it like? I know some areas are more sparsely populated than others.',
   -12.752256393432617),
  ("definitely, it's a great place to visit.  I think I'll go to japan next summer.",
   -13.0343017578125),
  ('ohhh, i like taiwans too. do you like japanese food?',
   -13.228504180908203),
  ("that's a great choice. i've never been to taiwans but it seems nice.",
   -13.988241195678711),
  ('Oh, I have never been there! do you like it? I have a favorite place in the world is the beach.',
   -14.026561737060547),
  ('that sounds like a nice place, i have never been to that place. do you like the country

In [18]:
from itertools import zip_longest
history = [
    "hi, where are you from",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah",
    "that's great, what is your favorite place?",
    "um maybe taiwan?"
]
pairs = zip_longest(*[iter(history)]*2)
print(list(pairs))

[('hi, where are you from', "i'm from the states! i'm from california originally but i live in utah now."), ('cool, i am also from utah', "that's great, what is your favorite place?"), ('um maybe taiwan?', None)]


In [5]:

blender_agent.topk

1